Data Preparation

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
abbreviation_map = {
    "u": "you",
    "btw": "by the way",
    "omg": "oh my god",
    "idk": "I don't know",
    "lol": "laughing out loud",
    "pls": "please",
    "thx": "thanks",
    "im": "I am",
    "dont": "do not",
    "cant": "cannot",
    "wont": "will not",
    # Add more as needed
}

In [4]:
#Changed the 'train_data' into 'raw_data' and changed 'test_data' into 'answer_data'

labelled_data = pd.read_json('train.json')
answer_data = pd.read_json('test.json')

print(labelled_data.head())
print(answer_data.head())

raw_labels = labelled_data['sentiments']

                                             reviews  sentiments
0  I bought this belt for my daughter in-law for ...           1
1  The size was perfect and so was the color.  It...           1
2  Fits and feels good, esp. for doing a swim rac...           1
3  These socks are absolutely the best. I take pi...           1
4  Thank you so much for the speedy delivery they...           1
                                             reviews
0  I bought 2 sleepers.  sleeper had holes in the...
1  I dare say these are just about the sexiest th...
2  everything about the transaction (price, deliv...
3  Not bad for just a shirt.  Very durable, and m...
4  These are truly wrinkle free and longer than t...


In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [5]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Expand abbreviations
    text = ' '.join([abbreviation_map.get(word, word) for word in text.split()])
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords and lemmatize
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

In [7]:
labelled_data['cleaned_reviews'] = labelled_data['reviews'].apply(clean_text)
answer_data['cleaned_reviews'] = answer_data['reviews'].apply(clean_text)

In [8]:
print(labelled_data.head())

                                             reviews  sentiments  \
0  I bought this belt for my daughter in-law for ...           1   
1  The size was perfect and so was the color.  It...           1   
2  Fits and feels good, esp. for doing a swim rac...           1   
3  These socks are absolutely the best. I take pi...           1   
4  Thank you so much for the speedy delivery they...           1   

                                     cleaned_reviews  
0         bought belt daughter inlaw christmas loved  
1            size perfect color looked like web page  
2  fits feels good esp swim race highly recommend...  
3  socks absolutely best take pilates classes hot...  
4  thank much speedy delivery came time rehearsal...  


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization: Basically divides the sentences into segments
tokenizer = Tokenizer()
tokenizer.fit_on_texts(labelled_data['cleaned_reviews']) #Indexes each token (each word is given a number). Index = 0 is for padding

# Convert text to sequences
raw_sequences = tokenizer.texts_to_sequences(labelled_data['cleaned_reviews'])
answer_sequences = tokenizer.texts_to_sequences(answer_data['cleaned_reviews'])

# Padding sequences
max_length = max(max(len(seq) for seq in raw_sequences), max(len(seq) for seq in answer_sequences)) #Finding the maximum sequence length from both the raw_data and answer_data
raw_final = pad_sequences(raw_sequences, maxlen=max_length, padding='post') #standardize the array
answer_final = pad_sequences(answer_sequences, maxlen=max_length, padding='post')

# Display the shape of padded sequences
print(f'Train padded shape: {raw_final.shape}')
print(f'Test padded shape: {answer_final.shape}')

Train padded shape: (7401, 518)
Test padded shape: (1851, 518)


In [10]:
raw_final

array([[  12,  288,  214, ...,    0,    0,    0],
       [   5,   39,   41, ...,    0,    0,    0],
       [  86,  323,    7, ...,    0,    0,    0],
       ...,
       [1309,  401,  217, ...,    0,    0,    0],
       [ 143,  957,  380, ...,    0,    0,    0],
       [ 230,  906,   97, ...,    0,    0,    0]])

In [11]:
answer_final

array([[  12,  116, 7406, ...,    0,    0,    0],
       [3212,  123, 5234, ...,    0,    0,    0],
       [ 172, 1543,   18, ...,    0,    0,    0],
       ...,
       [  15,  108,  211, ...,    0,    0,    0],
       [ 214,  132,   53, ...,    0,    0,    0],
       [3179,  917,  153, ...,    0,    0,    0]])

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Load GloVe word embeddings (download and extract glove.6B.100d.txt)
embedding_dim = 100 #as vectors of n real numbers
glove_file = 'C:/Users/User/Downloads/Telegram Desktop/glove.6B/glove.6B.100d.txt' #change this to your glove path
embeddings_index = {}

with open(glove_file, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Loaded {len(embeddings_index)} word vectors from GloVe.')

# Visualize the GloVe embeddings with t-SNE

# Step 1: Select a subset of words from the vocabulary (e.g., the first 100 words)
word_subset = list(tokenizer.word_index.keys())[:100]
word_vectors = np.array([embedding_matrix[tokenizer.word_index[word]] for word in word_subset])

# Step 2: Apply t-SNE to reduce dimensionality to 2D
tsne = TSNE(n_components=2, random_state=42)
word_vecs_2d = tsne.fit_transform(word_vectors)

# Step 3: Plot the words in 2D space
plt.figure(figsize=(10, 10))
for i, word in enumerate(word_subset):
    plt.scatter(word_vecs_2d[i, 0], word_vecs_2d[i, 1])
    plt.annotate(word, (word_vecs_2d[i, 0], word_vecs_2d[i, 1]), fontsize=9)
plt.title("2D Visualization of GloVe Embeddings (subset of vocabulary)")
plt.show()

Loaded 400000 word vectors from GloVe.


In [13]:
# Prepare the embedding matrix
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Map the words in the tokenizer's vocabulary to GloVe vectors
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector  # Words found in GloVe will use pretrained vectors

# Check the embedding matrix shape
print(f'Embedding matrix shape: {embedding_matrix.shape}')

Embedding matrix shape: (16366, 100)


In [14]:
max_length

518

In [15]:
#train test split the raw data
from sklearn.model_selection import train_test_split
labelled_train, labelled_test, label_train, label_test = train_test_split(
    raw_final, raw_labels, test_size=0.2, random_state=42)

In [22]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout


# Define your model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=True))
#CuDNN has very strict requirements to be able to use GPU, im putting it here to show the requirements, some of them are already default settings though
model.add(Bidirectional(LSTM(64,
                             activation='tanh',  # Default settings, to show
                             recurrent_activation='sigmoid',  # Default
                             return_sequences=True,  # Must be set to true for the first layer
                             recurrent_dropout=0,  # Must be 0 for cuDNN
                             unroll=False,  # Must be False for cuDNN
                             use_bias=True)))  # Default is True
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(4,
                             activation='tanh',  # Default settings, to show
                             recurrent_activation='sigmoid',  # Default
                             return_sequences=False,  # Set appropriately, not Default
                             recurrent_dropout=0,  # Must be 0 for cuDNN
                             unroll=False,  # Must be False for cuDNN
                             use_bias=True)))  # Default is True
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Print the model summary
model.summary()

snn_model_history = model.fit(labelled_train, label_train, batch_size=256, epochs=20, verbose=1, validation_split=0.2)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 518, 100)          1636600   
                                                                 
 bidirectional_8 (Bidirectio  (None, 518, 128)         84480     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 518, 128)          0         
                                                                 
 bidirectional_9 (Bidirectio  (None, 8)                4256      
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 8)                 0         
                                                                 
 dense_4 (Dense)             (None, 1)                

In [23]:
loss, acc = model.evaluate(labelled_test, label_test)

47/47 [==============================] - 5s 98ms/step - loss: 0.2617 - acc: 0.9041
